In [1]:
import pandas as pd

In [2]:
info = pd.read_csv("output/Walker_Basic_Info_Table.csv")
info = info[pd.notna(info['totalWalks'])]
info[:2]

,walkerId,totalWalks,firstWalk,lastWalk,signupTime,didDemandOrWalk,lifetime,averageLifetime,averageWalk,firstWalkAfterSignup,...,lastMonthWalks,activityTag,differentDistrictCount,activeLifeTime,totalActiveDayCount,walkFrequency,dailyWalkAverage,totalDemandCount,last30DaysDemandCount,weeklyAverageDemandCount
3,7f6a0cfd-b94d-4bb6-9db8-d7c28eb03e7b,12.0,2020-10-27 15:15:00.000,2021-01-25 16:49:21.821,2020-10-12 11:38:52.230,Yes,105.0,More,Less,15.0,...,0.0,NotActive,1.0,90 days 01:34:21.821000,0 days 08:54:24.522000,242.687598,32.334803,62.0,2.0,4.818714
4,fb487092-7521-4cb4-b102-a24cd337da54,1.0,2020-11-26 08:00:00.000,2020-11-26 08:00:00.000,2020-11-24 13:29:52.648,Yes,1.0,Less,Less,1.0,...,0.0,NotActive,1.0,0 days 00:00:00,0 days 00:43:45.778000,0.000000,32.904533,5.0,0.0,NaN


In [3]:
walks = pd.read_csv("data/walks.csv")
walks.rename(columns={"walkerid": "walkerId"}, inplace=True)
walks[:2]

,Unnamed: 0,ownerid,walkid,walkerId,walkingid,dogid,walkingtype,duration,checkintime,distance,district,neighborhood,endtime
0,0,eefa1cdf-041b-4738-a0f5-6e6647b28b1a,fdb4fdd3-19b7-4176-a2ee-817eeaba4f4d,09e59ced-1172-4744-a2bb-eb68ae83041c,e5b70802-a457-472a-9852-cba0889ef85a,1e68f0a4-101d-44fb-b9d6-add3277f1277,Customize,45,2019-12-06 07:00:00.000,2.826252,Beşiktaş,NaN,2019-12-06 08:06:06.853
1,1,b8ed3dd3-8a62-4189-85ee-b224913c64d6,e02af026-f364-40c4-9483-0adb4e449ed2,6007a0a8-d40b-4560-9ead-180c1646dae5,0d1f6547-7f63-4231-ace7-b23692102b70,fe49e24b-87ee-4512-b9e5-880e61021617,Customize,45,2020-03-03 07:30:00.000,1.241175,Kadıköy,NaN,2020-03-03 08:22:12.239


In [4]:
walkerServiceDates = dict()

for index,row in walks.iterrows():
    id = row["walkerId"]
    date = pd.to_datetime(row["endtime"])
    if(id in walkerServiceDates):
        walkerServiceDates[id].append([date, 0])
    else:
        walkerServiceDates[id] = [[date, 0]]
for walkerId,dates in walkerServiceDates.items():
    sorted_dates = sorted(dates)
    walkerServiceDates[walkerId] = sorted_dates

In [5]:
firstService = dict()
for id,date_list in walkerServiceDates.items():
    initial_walk = date_list[0][0]
    date_list[0][1] = 1
    for i in range(1,len(date_list)):
        if(id in firstService):
            if(date_list[i][0] < initial_walk + pd.Timedelta(days=30)):
                firstService[id] += 1
                date_list[0][1] = 1
        else:
            initial_walk = date_list[0][0]
            firstService[id] = 1
            date_list[0][1] = 1

new = pd.DataFrame()
new["walkerId"] = firstService.keys()
new["firstService"] = firstService.values()
new[:2]

,walkerId,firstService
0,09e59ced-1172-4744-a2bb-eb68ae83041c,12
1,6007a0a8-d40b-4560-9ead-180c1646dae5,8


In [6]:
returnFromLoss = dict()
for id,date_list in walkerServiceDates.items():
    for i in range(1,len(date_list)):
        prev_date = date_list[i-1][0]
        curr_date = date_list[i][0]

        if((curr_date - prev_date) > pd.Timedelta(days=60)):
            returnFromLoss[id] = 1
            date_list[i][1] = 2
            for j in range(i+1, len(date_list)):
                if(date_list[j][0] < date_list[i][0] + pd.Timedelta(days=30)):
                    returnFromLoss[id] += 1
                    date_list[j][1] = 2
                else:
                    break

new["returnFromLoss"] = new["walkerId"].map(returnFromLoss).fillna(0).astype(int)
new[:2]


,walkerId,firstService,returnFromLoss
0,09e59ced-1172-4744-a2bb-eb68ae83041c,12,5
1,6007a0a8-d40b-4560-9ead-180c1646dae5,8,23


In [7]:
returnFromPassive = dict()
for id,date_list in walkerServiceDates.items():
    for i in range(1,len(date_list)):
        prev_date = date_list[i-1][0]
        curr_date = date_list[i][0]

        if(pd.Timedelta(days=30) < (curr_date - prev_date) < pd.Timedelta(days=60)):
            returnFromPassive[id] = 1
            date_list[i][1] = 3
            for j in range(i+1, len(date_list)):
                if(date_list[j][0] < date_list[i][0] + pd.Timedelta(days=30)):
                    returnFromPassive[id] += 1
                    date_list[i][1] = 3
                else:
                    break

new["returnFromPassive"] = new["walkerId"].map(returnFromPassive).fillna(0).astype(int)
new[:2]

,walkerId,firstService,returnFromLoss,returnFromPassive
0,09e59ced-1172-4744-a2bb-eb68ae83041c,12,5,2
1,6007a0a8-d40b-4560-9ead-180c1646dae5,8,23,0


In [8]:
retained = dict()

for id, date_list in walkerServiceDates.items():
    for i in range(len(date_list)):
        if(date_list[i][1] == 0):
            if id in retained:
                retained[id] += 1
                date_list[i][1] = 4
            else:
                retained[id] = 1
                date_list[i][1] = 4

new["retained"] = new["walkerId"].map(retained).fillna(0).astype(int)
new[:2]

,walkerId,firstService,returnFromLoss,returnFromPassive,retained
0,09e59ced-1172-4744-a2bb-eb68ae83041c,12,5,2,91
1,6007a0a8-d40b-4560-9ead-180c1646dae5,8,23,0,250


In [9]:
new.to_csv("output/Walker_Basic_Cohort_Table_counts.csv", index=False)

In [10]:
rows = list()
for key,date_list in walkerServiceDates.items():
    for i in range(len(date_list)):
        year_month = date_list[i][0].strftime('%Y-%m')
        walker_id = key
        service_type = date_list[i][1]
        typo = [0,0,0,0]
        typo[service_type-1] = 1        
        
        rows.append([year_month, walker_id] + typo) 

new = pd.DataFrame(rows, columns=["yearMonth", "walkerId", "firstService","returnFromLoss","returnFromPassive","retained"])
new[:2]



,yearMonth,walkerId,firstService,returnFromLoss,returnFromPassive,retained
0,2019-09,09e59ced-1172-4744-a2bb-eb68ae83041c,1,0,0,0
1,2019-09,09e59ced-1172-4744-a2bb-eb68ae83041c,0,0,0,1


In [11]:
new = new.sort_values(by="yearMonth")
new.reset_index(inplace=True, drop=True)
new[:2]

,yearMonth,walkerId,firstService,returnFromLoss,returnFromPassive,retained
0,2019-01,5f54e91b-52cc-47e7-82b6-f465b2993d50,0,0,0,1
1,2019-01,5f54e91b-52cc-47e7-82b6-f465b2993d50,0,0,0,1


In [12]:
new.to_csv("output/Walker_Basic_Cohort_Table.csv", index=False)